# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [3]:
!pip install psycopg2

In [115]:
import os
import glob
import psycopg2
import numpy as np
import pandas as pd
from sql_queries import *

In [2]:
song_table_insert

'\n    INSERT INTO songs\n    (song_id, title, artist_id, year, duration)\n    VALUES (%d, %s, %s, %d, %d)\n'

In [391]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
conn.set_session(autocommit=True)
cur = conn.cursor()

In [394]:
created_tables = ("""
    SELECT table_name 
    FROM information_schema.tables 
    WHERE table_schema='public';
""")

In [397]:
print('\n-----------------------------------------')
print('  Created tables')
print('-----------------------------------------')
cur.execute(created_tables)
for table in cur.fetchall():
    print(table[0])


-----------------------------------------
  Created tables
-----------------------------------------
songplays
users
songs
artists
time


In [398]:
conn.close()

In [377]:
query = ("""
    SELECT 
        (SELECT COUNT(*) FROM songplays) AS songplays_count
        , (SELECT COUNT(*) FROM time) AS time_count    
        , (SELECT COUNT(*) FROM users) AS users_count        
        , (SELECT COUNT(*) FROM songs) AS songs_count
        , (SELECT COUNT(*) FROM artists) AS artists_count        
""")

tables = ['songplays', 'time', 'users', 'songs', 'artists']
cur.execute(query)
for count in cur.fetchall()[0]:
    print(f'{tables.pop(0)}: {count}')

songplays: 6813
time: 6813
users: 96
songs: 71
artists: 69


In [390]:
conn.close()

In [338]:
cur.execute("SELECT * FROM artists")

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

In [5]:
# Data directory
data_dir = 'D:\\udacity\\Data Engineering\\project 1\\data\\'

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [291]:
data_dir = 'D:/udacity/Data Engineering/project 1/data/'
song_files = get_files(data_dir +'song_data')

In [7]:
filepath = song_files[0]

In [62]:
df = pd.read_json(filepath, lines=True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [9]:
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']]
song_data

,song_id,title,artist_id,year,duration
0,SOMZWCG12A8C13C480,I Didn't Mean To,ARD7TVE1187B99BFB1,0,218.93179


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [226]:
song_table_insert = ("""
    INSERT INTO songs
    (song_id, title, artist_id, year, duration)
    VALUES (%s, %s, %s, %s, %s)
    ON CONFLICT (song_id) DO NOTHING;
""")

In [227]:
song_data

,song_id,title,artist_id,year,duration
0,SOMZWCG12A8C13C480,I Didn't Mean To,ARD7TVE1187B99BFB1,0,218.93179


In [ ]:
cur.execute(song_table_insert, tuple(song_data.values[0]))
conn.commit()

In [240]:
cur.execute()
cur.fetchall()

[('SOMZWCG12A8C13C480',
  "I Didn't Mean To",
  'ARD7TVE1187B99BFB1',
  0,
  Decimal('218.93179'))]

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [32]:
df

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0


In [66]:
# Replace NaN by None in order to have NULL values is db (instead of 'NaN')
df = df.where(pd.notnull(df), None)

In [70]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']]
artist_data

,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
0,ARD7TVE1187B99BFB1,Casual,California - LA,None,None


#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [185]:
artist_table_insert = ("""
    INSERT INTO artists
    (artist_id, name, location, latitude, longitude)
    VALUES (%s, %s, %s, %s, %s)
    ON CONFLICT (artist_id) DO NOTHING;
""")

In [186]:
artist_data

,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
0,ARD7TVE1187B99BFB1,Casual,California - LA,None,None


In [241]:
cur.execute(artist_table_insert, tuple(artist_data.values[0]))
conn.commit()

In [242]:
cur.execute("SELECT * FROM artists;")
cur.fetchall()

[('ARD7TVE1187B99BFB1', 'Casual', 'California - LA', None, None)]

In [76]:
cur.execute("TRUNCATE TABLE artists;")

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [82]:
log_files = get_files(data_dir +'log_data')

In [84]:
filepath = log_files[0]
filepath

'D:\\udacity\\Data Engineering\\project 1\\data\\log_data\\2018\\11\\2018-11-01-events.json'

In [142]:
log_df = pd.read_json(filepath, lines=True)
log_df.head(3)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1540919166796,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1540344794796,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [253]:
# Extract only NextSong rows
log_source_df = log_df[log_df['page'] == 'NextSong']

In [305]:
# Create time_df dataframe with all time dimension data converting unix timestamp into datetime
time_df = pd.DataFrame()
time_df['start_time'] = pd.to_datetime(log_source_df['ts'].drop_duplicates(), unit='ms')
time_df['hour'] = time_df['start_time'].dt.hour
time_df['day'] = time_df['start_time'].dt.day
time_df['week'] = time_df['start_time'].dt.week
time_df['month'] = time_df['start_time'].dt.month
time_df['year'] = time_df['start_time'].dt.year
time_df['weekday'] = time_df['start_time'].dt.weekday
time_df

,start_time,hour,day,week,month,year,weekday
2,2018-11-01 21:01:46.796,21,1,44,11,2018,3
4,2018-11-01 21:05:52.796,21,1,44,11,2018,3
5,2018-11-01 21:08:16.796,21,1,44,11,2018,3
6,2018-11-01 21:11:13.796,21,1,44,11,2018,3
7,2018-11-01 21:17:33.796,21,1,44,11,2018,3
8,2018-11-01 21:24:53.796,21,1,44,11,2018,3
9,2018-11-01 21:28:54.796,21,1,44,11,2018,3
10,2018-11-01 21:42:00.796,21,1,44,11,2018,3
12,2018-11-01 21:52:05.796,21,1,44,11,2018,3
13,2018-11-01 21:55:25.796,21,1,44,11,2018,3


In [301]:
# filter by NextSong action
df = log_df[log_df['page'] == 'NextSong']

# convert timestamp column to datetime
t = pd.to_datetime(df['ts'], unit='ms')
    
# insert time data records
time_data = (
    t,
    t.dt.hour,
    t.dt.day,
    t.dt.week,
    t.dt.month,
    t.dt.year,
    t.dt.weekday
)

time_data

(2    2018-11-01 21:01:46.796
 4    2018-11-01 21:05:52.796
 5    2018-11-01 21:08:16.796
 6    2018-11-01 21:11:13.796
 7    2018-11-01 21:17:33.796
 8    2018-11-01 21:24:53.796
 9    2018-11-01 21:28:54.796
 10   2018-11-01 21:42:00.796
 12   2018-11-01 21:52:05.796
 13   2018-11-01 21:55:25.796
 14   2018-11-01 22:23:14.796
 Name: ts, dtype: datetime64[ns],
 2     21
 4     21
 5     21
 6     21
 7     21
 8     21
 9     21
 10    21
 12    21
 13    21
 14    22
 Name: ts, dtype: int64,
 2     1
 4     1
 5     1
 6     1
 7     1
 8     1
 9     1
 10    1
 12    1
 13    1
 14    1
 Name: ts, dtype: int64,
 2     44
 4     44
 5     44
 6     44
 7     44
 8     44
 9     44
 10    44
 12    44
 13    44
 14    44
 Name: ts, dtype: int64,
 2     11
 4     11
 5     11
 6     11
 7     11
 8     11
 9     11
 10    11
 12    11
 13    11
 14    11
 Name: ts, dtype: int64,
 2     2018
 4     2018
 5     2018
 6     2018
 7     2018
 8     2018
 9     2018
 10    2018
 12    2018

In [319]:
pd.to_datetime(df['ts'], unit='ms')

2    2018-11-01 21:01:46.796
4    2018-11-01 21:05:52.796
5    2018-11-01 21:08:16.796
6    2018-11-01 21:11:13.796
7    2018-11-01 21:17:33.796
8    2018-11-01 21:24:53.796
9    2018-11-01 21:28:54.796
10   2018-11-01 21:42:00.796
12   2018-11-01 21:52:05.796
13   2018-11-01 21:55:25.796
14   2018-11-01 22:23:14.796
Name: ts, dtype: datetime64[ns]

In [326]:
t = df['ts'].apply(lambda x: pd.Timestamp(1513393355.5, unit='s'))

In [331]:
t.apply(lambda x: x.isocalendar()[0])

2     2017
4     2017
5     2017
6     2017
7     2017
8     2017
9     2017
10    2017
12    2017
13    2017
14    2017
Name: ts, dtype: int64

In [329]:
pd.Timestamp(1513393355.5, unit='s').isocalendar()

(2017, 50, 6)

In [333]:
# filter by NextSong action
df = log_df[log_df['page'] == 'NextSong']

# convert timestamp column to datetime
t = pd.to_datetime(df['ts'], unit='ms')
    
# we use isocalendar function because Series.dt.week has been deprecated    
time_df = pd.DataFrame({
    'start_time': t,
    'hour': t.dt.hour,
    'day': t.dt.day,
    'week': t.apply(lambda x: x.isocalendar()[1]),
    'month': t.dt.month,
    'year': t.dt.year,
    'weekday': t.dt.weekday
})

#for i, row in time_df.iterrows():
#    cur.execute(time_table_insert, list(row))

time_df

,start_time,hour,day,week,month,year,weekday
2,2018-11-01 21:01:46.796,21,1,44,11,2018,3
4,2018-11-01 21:05:52.796,21,1,44,11,2018,3
5,2018-11-01 21:08:16.796,21,1,44,11,2018,3
6,2018-11-01 21:11:13.796,21,1,44,11,2018,3
7,2018-11-01 21:17:33.796,21,1,44,11,2018,3
8,2018-11-01 21:24:53.796,21,1,44,11,2018,3
9,2018-11-01 21:28:54.796,21,1,44,11,2018,3
10,2018-11-01 21:42:00.796,21,1,44,11,2018,3
12,2018-11-01 21:52:05.796,21,1,44,11,2018,3
13,2018-11-01 21:55:25.796,21,1,44,11,2018,3


In [332]:
# filter by NextSong action
df = log_df[log_df['page'] == 'NextSong']

# convert timestamp column to datetime
t = pd.to_datetime(df['ts'], unit='ms')
    
time_df = pd.DataFrame({
    'start_time': t,
    'hour': t.dt.hour,
    'day': t.dt.day,
    'week': t.apply(lambda x: x.isocalendar()[1]),
    'month': t.dt.month,
    'year': t.apply(lambda x: x.isocalendar()[0]),
    'weekday': t.apply(lambda x: x.isocalendar()[2])
})

#for i, row in time_df.iterrows():
#    cur.execute(time_table_insert, list(row))

time_df

,start_time,hour,day,week,month,year,weekday
2,2018-11-01 21:01:46.796,21,1,44,11,2018,4
4,2018-11-01 21:05:52.796,21,1,44,11,2018,4
5,2018-11-01 21:08:16.796,21,1,44,11,2018,4
6,2018-11-01 21:11:13.796,21,1,44,11,2018,4
7,2018-11-01 21:17:33.796,21,1,44,11,2018,4
8,2018-11-01 21:24:53.796,21,1,44,11,2018,4
9,2018-11-01 21:28:54.796,21,1,44,11,2018,4
10,2018-11-01 21:42:00.796,21,1,44,11,2018,4
12,2018-11-01 21:52:05.796,21,1,44,11,2018,4
13,2018-11-01 21:55:25.796,21,1,44,11,2018,4


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [221]:
time_table_insert = ("""
    INSERT INTO time
    (start_time, hour, day, week, month, year, weekday)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    ON CONFLICT (start_time) DO NOTHING;
""")

In [245]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [148]:
log_df.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1540919166796,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39


In [169]:
user_df = log_df[['userId', 'firstName', 'lastName', 'gender', 'level']].drop_duplicates()
user_df

,userId,firstName,lastName,gender,level
0,39,Walter,Frye,M,free
1,8,Kaylee,Summers,F,free
10,10,Sylvie,Cruz,F,free
11,26,Ryan,Smith,M,free
14,101,Jayden,Fox,M,free


In [307]:
log_df.shape

(15, 18)

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [181]:
user_table_insert = ("""
    INSERT INTO users
    (user_id, first_name, last_name, gender, level)
    VALUES (%s, %s, %s, %s, %s)
    ON CONFLICT (user_id) DO NOTHING;
""")

In [246]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
# Implement the song_select query in sql_queries.py to find 
# the song ID and artist ID based on the title, artist name, and duration of a song.



In [ ]:
SELECT A.song_id, A.artist_id \
FROM songs AS A \
INNER JOIN artists AS B ON A.artist_id = B.artist_id \
WHERE A.title = 'I Didn''t Mean To' \
    AND B.name = 'Casual' \
    AND A.duration = 218.93179

In [277]:
song_select = ("""
    SELECT A.song_id, A.artist_id 
    FROM songs AS A
    INNER JOIN artists AS B ON A.artist_id = B.artist_id
    WHERE A.title = %s
        AND B.name = %s
        AND A.duration = %s;
""")

In [280]:
time_source_df

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Quem Quiser Encontrar O Amor,200,1541106496796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Eriatarka,200,1541106673796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Becoming Insane,200,1541107053796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
8,Blue October / Imogen Heap,Logged In,Kaylee,F,7,Summers,241.39710,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Congratulations,200,1541107493796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
9,Girl Talk,Logged In,Kaylee,F,8,Summers,160.15628,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Once again,200,1541107734796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
10,Black Eyed Peas,Logged In,Sylvie,F,0,Cruz,214.93506,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",PUT,NextSong,1540266185796,9,Pump It,200,1541108520796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",10
12,Fall Out Boy,Logged In,Ryan,M,1,Smith,200.72444,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1541016707796,169,Nobody Puts Baby In The Corner,200,1541109125796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
13,M.I.A.,Logged In,Ryan,M,2,Smith,233.71710,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1541016707796,169,Mango Pickle Down River (With The Wilcannia Mob),200,1541109325796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26


In [276]:
df

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,None,None,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.932,0


In [287]:
songplay_table_insert = ("""
    INSERT INTO songplays
    (start_time, user_id, level, song_id, artist_id, session_id, location, user_agent)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    ON CONFLICT (start_time) DO NOTHING;
""")

In [ ]:
songplay_data = (row.start)

In [261]:
log_source_df['start_time'] = pd.to_datetime(log_source_df['ts'], unit='ms')

<ipython-input-261-40c5f7c2afa5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  log_source_df['start_time'] = pd.to_datetime(log_source_df['ts'], unit='ms')


In [285]:
songplay_table_insert

'\n    INSERT INTO songplays\n    (start_time, user_id, level, song_id, artist_id, session_id, location, user_agent)\n    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)\n    ON CONFLICT (song_id) DO NOTHING;\n'

In [289]:
for index, row in log_source_df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        song_id, artist_id = results
    else:
        song_id, artist_id = None, None
        
    print (song_id, artist_id)

    # insert songplay record
    songplay_data = (row.start_time, row.userId, row.level, song_id, artist_id, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
 

None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None


Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [295]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.